In [31]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import re
from gensim.models import FastText
import os
from konlpy.tag import Okt




In [32]:
# 각자의 Ctee 폴더 경로
my_dir = '/home/wooksbaby/boostcamp6th/'

In [33]:
items_df = pd.read_json(my_dir + "20240308-data/items.json")  # 콘텐츠
events_df = pd.read_json(my_dir + "20240308-data/event_projects.json")  # 행사/이벤트
cats_df = pd.read_json(my_dir + "20240219-data/categories_events.json")  # 이벤트 분류기준
stores_df = pd.read_json(my_dir + "20240219-data/stores.json")  # 크리에이터 스토어

funnels_df = pd.read_json(my_dir + "20240221-data/funnels.json")  # 유입경로
orders_df = pd.read_json(my_dir + "20240308-data/orders_items.json")  # 구매데이터
subs_df = pd.read_json(my_dir + "20240221-data/user_subscribes.json")  # 구독정보

items_tag_df = pd.read_json(my_dir + "CteeEDA/tag_output_items.json")  # 구독정보
stores_tag_df = pd.read_json(my_dir + "CteeEDA/tag_output_stores.json")  # 구독정보


In [34]:
items_df = items_df.drop_duplicates()
stores_df = stores_df.drop_duplicates()
items_tag_df = items_tag_df.drop_duplicates()
stores_tag_df = stores_tag_df.drop_duplicates()

In [35]:
items_tag_df.columns
stores_tag_df.columns
items_df.columns
stores_df.columns


Index(['id', 'title', 'content', 'alias', 'view_count', 'sales_volume',
       'donation_count', 'created_at', 'updated_at'],
      dtype='object')

In [36]:
items_df = items_df.rename(columns={
    'id': 'item_id'
})
stores_df = stores_df.rename(columns={
    'id': 'store_id'
})
items_tag_df = items_tag_df.rename(columns={
    'id': 'item_id'
})

In [37]:
items_tag_df.columns


Index(['item_id', 'tags', 'category', 'categories'], dtype='object')

In [38]:
merged_items_df = pd.merge(items_df, items_tag_df, on='item_id',how='left')
merged_stores_df = pd.merge(stores_df, stores_tag_df, on='store_id',how='left')

In [39]:
merged_items_df.columns

Index(['item_id', 'store_id', 'simple_contents', 'content', 'story',
       'product_category_type', 'currency_code', 'price', 'discount_price',
       'discount_started_at', 'discount_ended_at', 'is_adult', 'order_number',
       'sales_count', 'view_count', 'comment_count', 'created_at',
       'updated_at', 'tags', 'category', 'categories'],
      dtype='object')

In [40]:
merged_stores_df.columns

Index(['store_id', 'title', 'content', 'alias', 'view_count', 'sales_volume',
       'donation_count', 'created_at', 'updated_at', 'tags', 'category'],
      dtype='object')

In [41]:
selected_columns_items_df= merged_items_df[['item_id','store_id','content','price','is_adult','sales_count','view_count','tags','category']]

In [42]:
selected_columns_stores_df= merged_stores_df[['store_id','content','view_count','sales_volume','tags','category']]

In [43]:
selected_columns_items_df = selected_columns_items_df.rename(columns={
    'id': 'item_id',
    'simple_contents': 'item_simple_contents',
    'content': 'item_content',
    'story': 'item_story',
    'category' : 'item_category'
})

In [44]:
selected_columns_stores_df = selected_columns_stores_df.rename(columns={
    'content': 'store_content',
    'view_count': 'store_view_count',
    'sales_volume': 'store_sales_volume',
    'tags': 'store_tags',
    'category': 'store_category'
})

In [45]:
selected_columns_items_df

,item_id,store_id,item_content,price,is_adult,sales_count,view_count,tags,item_category
0,3,2,"안녕하세요, 그림 스피드페인팅 유튜버 노넴입니다 :)\n\n원하시는 컨셉에 맞춰 일...",20000,0,0,76,#일러스트#아트#그림,"미술 ∙ 디자인, 영상 ∙ 사진"
1,4,2,"안녕하세요, 그림 스피드페인팅 유튜버 노넴입니다 :)\n\n원하시는 컨셉에 맞춰 일...",25000,0,0,64,#일러스트#아트#그림,"미술 ∙ 디자인, 영상 ∙ 사진"
2,5,2,"안녕하세요, 그림 스피드페인팅 유튜버 노넴입니다 :)\n\n원하시는 컨셉에 맞춰 일...",30000,0,0,30,#일러스트#아트#그림,"미술 ∙ 디자인, 영상 ∙ 사진"
3,6,2,"안녕하세요, 그림 스피드페인팅 유튜버 노넴입니다 :)\n\n원하시는 컨셉에 맞춰 일...",50000,0,0,40,#일러스트#아트#그림,"미술 ∙ 디자인, 영상 ∙ 사진"
4,7,3,"안녕하세요, 관상가 화인입니다 :) \n\n비대면 시대에 꼭 어울리는 관상 콘텐츠를...",20000,0,9,415,#관상가#관상#비대면#상담,"타로 ∙ 사주 ∙ 관상, 심리 ∙ 상담"
...,...,...,...,...,...,...,...,...,...
18023,22217,7165,,6800,0,0,0,#미포함#확성기#아이콘,기타
18024,22218,7165,,4800,0,0,1,#미포함#손가락숫자,기타
18025,22219,7165,,4800,0,0,0,#미포함#여자,기타
18026,22220,7165,,4800,0,0,0,#미포함#남자,기타


In [46]:
# items_df['sales_value'] = items_df['sales_count'] * items_df['price']
# sorted_items = items_df.sort_values(by='sales_value', ascending=False)

# # 스토어의 총 판매량 ('sales_volume') 계산 후 내림차순으로 정렬
# sorted_stores = stores_df.sort_values(by='sales_volume', ascending=False)

# # 결과 출력 (여기서는 상위 5개 항목 예시)
# print(sorted_items.head(20))
# print(sorted_stores.head(20))

In [47]:
selected_columns_stores_df.to_json('selected_columns_stores_df_with_tags.jsonl', orient='records',lines=True,force_ascii=False)
selected_columns_items_df.to_json('selected_columns_items_df_with_tags.jsonl', orient='records',lines=True,force_ascii=False)


In [48]:
selected_columns_stores_df.columns

Index(['store_id', 'store_content', 'store_view_count', 'store_sales_volume',
       'store_tags', 'store_category'],
      dtype='object')

In [49]:
selected_columns_items_df.columns

Index(['item_id', 'store_id', 'item_content', 'price', 'is_adult',
       'sales_count', 'view_count', 'tags', 'item_category'],
      dtype='object')